In [1]:
import os
from PIL import Image
import shutil
import random
import yaml
import torch
from ultralytics import YOLO

In [2]:
# Configura el dispositivo y notifica sobre la disponibilidad de CUDA
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"{'CUDA está disponible. Usando CUDA.' if torch.cuda.is_available() else 'CUDA no está disponible. Usando CPU.'}")

CUDA está disponible. Usando CUDA.


In [3]:
class DatasetValidator:
    def __init__(self, main_path):
        """
        Initializes the DatasetValidator with the paths to the images and labels directories.

        Parameters:
        main_path (str): The path to the folder containing all files.
        """
        self.main_path = main_path
        self.images_path = os.path.join(main_path, 'images')
        self.labels_path = os.path.join(main_path, 'labels')

    def check_matching_files(self):
        """
        Checks if each label file (.txt) in the 'labels' folder has a corresponding
        image file in the 'images' folder. If there are any mismatches, it will print
        which images or labels are missing.

        Returns:
        None
        """
        # Get filenames from the directories
        image_files = set(os.listdir(self.images_path))
        label_files = set(os.listdir(self.labels_path))
        
        # Filter only the filenames without extensions
        image_names = set(os.path.splitext(image)[0] for image in image_files)
        label_names = set(os.path.splitext(label)[0] for label in label_files)
        
        # Compare the filenames
        missing_images = label_names - image_names
        missing_labels = image_names - label_names
        
        if missing_images:
            print(f"Missing images for the following labels: {', '.join(missing_images)}")
        else:
            print("All labels have corresponding images.")
        
        if missing_labels:
            print(f"Missing labels for the following images: {', '.join(missing_labels)}")
        else:
            print("All images have corresponding labels.")

    def validate_images(self):
        """
        Checks if the image files in the 'images' folder can be opened without errors.
        If there are any corrupt files, it will print which images are corrupt.

        Returns:
        None
        """
        # Validate image files
        corrupt_images = []
        image_files = os.listdir(self.images_path)
        for image_file in image_files:
            image_path = os.path.join(self.images_path, image_file)
            try:
                with Image.open(image_path) as img:
                    img.verify()  # Verify that it is, in fact, an image
            except (IOError, SyntaxError) as e:
                corrupt_images.append(image_file)
        
        if corrupt_images:
            print(f"Corrupt images detected: {', '.join(corrupt_images)}")
        else:
            print("No corrupt images found.")

    def validate_labels(self):
        """
        Checks if the label files in the 'labels' folder can be read without errors.
        If there are any corrupt files, it will print which labels are corrupt.

        Returns:
        None
        """
        # Validate label files
        corrupt_labels = []
        label_files = os.listdir(self.labels_path)
        for label_file in label_files:
            label_path = os.path.join(self.labels_path, label_file)
            try:
                with open(label_path, 'r') as file:
                    file.read()  # Attempt to read the file
            except (IOError, UnicodeDecodeError) as e:
                corrupt_labels.append(label_file)
        
        if corrupt_labels:
            print(f"Corrupt labels detected: {', '.join(corrupt_labels)}")
        else:
            print("No corrupt labels found.")

    def filter_label_lines(self, ids_to_keep):
        """
        Filters the lines of each label file (.txt) in the 'labels' folder to keep only
        those lines where the first value (id) is in the specified list of ids.

        Parameters:
        ids_to_keep (list of int): The list of ids to keep.

        Returns:
        None
        """
        # Create a backup of the original dataset folder
        backup_path = self.main_path + '_TMP'
        shutil.copytree(self.main_path, backup_path)
        
        # Read each label file in the backup folder, filter lines and rewrite the file
        for label_file in os.listdir(os.path.join(backup_path, 'labels')):
            original_label_path = os.path.join(backup_path, 'labels', label_file)
            filtered_lines = []
            with open(original_label_path, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    parts = line.strip().split()
                    if int(parts[0]) in ids_to_keep:
                        filtered_lines.append(line)
            with open(original_label_path, 'w') as file:
                file.writelines(filtered_lines)

    def remove_empty_labels_and_corresponding_images(self, main_path):
        """
        Removes empty label files (.txt) from the 'labels' folder and their corresponding
        images from the 'images' folder in the specified main path.

        Parameters:
        main_path (str): The path to the base folder containing 'images' and 'labels' subfolders.

        Returns:
        None
        """
        images_path = os.path.join(main_path, 'images')
        labels_path = os.path.join(main_path, 'labels')


        empty_label_files = []
        label_files = os.listdir(labels_path)
        for label_file in label_files:
            label_path = os.path.join(labels_path, label_file)
            try:
                with open(label_path, 'r') as file:
                    content = file.read()
                    if not content.strip():
                        empty_label_files.append(label_file)
            except (IOError, UnicodeDecodeError) as e:
                pass
        
        if empty_label_files:
            print(f"Empty label files detected: {', '.join(empty_label_files)}")
            for empty_label in empty_label_files:
                image_file = empty_label.split('.')[0] + '.jpg'
                image_path = os.path.join(images_path, image_file)
                if os.path.exists(image_path):
                    os.remove(image_path)
                    print(f"Corresponding image {image_file} deleted. {image_path}")
                else:
                    print(f"Corresponding image {image_file} not found.")
            # Remove empty label files
            for empty_label in empty_label_files:
                label_path = os.path.join(labels_path, empty_label)
                os.remove(label_path)
                print(f"Empty label file {empty_label} deleted. {label_path}")
        else:
            print("No empty label files found.")

    def remove_TMP_folder(self, backup_folder):
        """
        Removes the specified backup folder created during operations.

        Parameters:
        folder_name (str): The name of the backup folder to be removed.

        Returns:
        None
        """
        if os.path.exists(backup_folder):
            shutil.rmtree(backup_folder)
            print("_TMP folder deleted.")
        else:
            print("_TMP folder not found.", backup_folder)

    def split_dataset(self, from_path, to_path, to_folder, train_ratio=0.8):
        """
        Splits the dataset into training and validation sets based on the provided ratio.

        Parameters:
        from_path (str): The path to the base folder containing 'images' and 'labels' subfolders.
        to_path (str): The path to the final folder containing 'images' and 'labels' subfolders.
        to_folder (str): Name of the final folder.
        train_ratio (float): The ratio of the dataset to be allocated to training (default is 0.8).

        Returns:
        train_folder (str): Path to the training dataset folder.
        validation_folder (str): Path to the validation dataset folder.
        """
        # Create a new folder for the split dataset
        to_path = os.path.join(to_path, to_folder)
        os.makedirs(to_path, exist_ok=True)
        to_path = os.path.join(to_path, 'dataset')
        os.makedirs(to_path, exist_ok=True)
        # Create folders for training and validation sets
        train_folder = os.path.join(to_path, 'train')
        validation_folder = os.path.join(to_path, 'validation')
        os.makedirs(train_folder, exist_ok=True)
        os.makedirs(validation_folder, exist_ok=True)
        train_images_folder = os.path.join(to_path, 'train', 'images')
        train_labels_folder = os.path.join(to_path, 'train', 'labels')
        os.makedirs(train_images_folder, exist_ok=True)
        os.makedirs(train_labels_folder, exist_ok=True)
        validation_images_folder = os.path.join(to_path, 'validation', 'images')
        validation_labels_folder = os.path.join(to_path, 'validation', 'labels')
        os.makedirs(validation_images_folder, exist_ok=True)
        os.makedirs(validation_labels_folder, exist_ok=True)

        # Define images and labels paths
        images_path = os.path.join(from_path, 'images')
        labels_path = os.path.join(from_path, 'labels')

        # Get list of image files
        image_files = os.listdir(images_path)
        num_images = len(image_files)

        # Shuffle the list of image files
        random.shuffle(image_files)

        # Split the dataset based on the ratio
        num_train = int(num_images * train_ratio)
        train_images = image_files[:num_train]
        validation_images = image_files[num_train:]

        # Copy images and labels to train folder
        for image_file in train_images:
            image_src = os.path.join(images_path, image_file)
            label_file = os.path.splitext(image_file)[0] + '.txt'
            label_src = os.path.join(labels_path, label_file)
            shutil.copy(image_src, train_images_folder)
            shutil.copy(label_src, train_labels_folder)

        # Copy images and labels to validation folder
        for image_file in validation_images:
            image_src = os.path.join(images_path, image_file)
            label_file = os.path.splitext(image_file)[0] + '.txt'
            label_src = os.path.join(labels_path, label_file)
            shutil.copy(image_src, validation_images_folder)
            shutil.copy(label_src, validation_labels_folder)

        return train_folder, validation_folder

In [4]:
validator = DatasetValidator('datasets/bioview')
validator.check_matching_files()
validator.validate_images()
validator.validate_labels()

All labels have corresponding images.
All images have corresponding labels.
No corrupt images found.
No corrupt labels found.


Podemos filtrar el dataset para solo quedarnos con las etiquetas que queramos trabajar. Lo haremos filtrando su ID. En este caso, solo queremos las lagartijas con ID 0.

In [5]:
validator.filter_label_lines([0])

Además también será conveniente comprobar que no haya etiquetas en blanco que no aportan nada al modelo.

In [6]:
validator.remove_empty_labels_and_corresponding_images('datasets/bioview_TMP')

Empty label files detected: img_20230626_010005.txt
Corresponding image img_20230626_010005.jpg deleted. datasets/bioview_TMP/images/img_20230626_010005.jpg
Empty label file img_20230626_010005.txt deleted. datasets/bioview_TMP/labels/img_20230626_010005.txt


In [7]:
validator.split_dataset(from_path='datasets/bioview_TMP',
                        to_path='datasets', to_folder='bioview-lizards_TRAIN',
                        train_ratio=0.8)

('datasets/bioview-lizards_TRAIN/dataset/train',
 'datasets/bioview-lizards_TRAIN/dataset/validation')

Eliminamos la carpeta temporal de *backup* que hemos hecho.

In [8]:
validator.remove_TMP_folder('datasets/bioview_TMP')

_TMP folder deleted.


Ahora que ya todo está estructurado cómo queremos creamos y guardamos el archivo yaml que contendrá las rutas y etiquetas del modelo.

In [9]:
train_path = os.path.join(os.getcwd(), 'datasets/bioview-lizards_TRAIN')
yaml_name = 'config.yaml'
yaml_path = os.path.join(train_path, yaml_name)

yaml_content = {
    'path': train_path,
    'train': "dataset/train/images",
    'val': "dataset/validation/images",
    'names': {
        0: "Lizard"
    }
}

# Crear o modificar el archivo YAML
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(yaml_content, yaml_file, default_flow_style=False)

print(f"Archivo YAML creado o modificado en '{yaml_path}'.")

Archivo YAML creado o modificado en '/home/qcienmed/mmr689/yolo-consumptions/datasets/bioview-lizards_TRAIN/config.yaml'.


In [10]:
model = YOLO("yolov8n.yaml")
results = model.train(data=yaml_path,
                      epochs=1000,
                      patience=100,
                      dropout= 0.2,
                      single_cls=False,
                      val=False,
                      device=device,
                      verbose=False,
                      project=train_path,
                      name='run/train'
                      )

New https://pypi.org/project/ultralytics/8.2.24 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.47 🚀 Python-3.8.10 torch-2.3.0+cu121 CUDA:1 (NVIDIA RTX A6000, 48677MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/home/qcienmed/mmr689/yolo-consumptions/datasets/bioview-lizards_TRAIN/config.yaml, epochs=1000, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=1, workers=8, project=/home/qcienmed/mmr689/yolo-consumptions/datasets/bioview-lizards_TRAIN, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=False, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream

/home/qcienmed/mmr689/yolo-consumptions/.venv/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
train: Scanning /home/qcienmed/mmr689/yolo-consumptions/datasets/bioview-lizards_TRAIN/dataset/train/labels... 580 images, 0 backgrounds, 0 corrupt: 100%|██████████| 580/580 [00:00<00:00, 1161.48it/s]


train: New cache created: /home/qcienmed/mmr689/yolo-consumptions/datasets/bioview-lizards_TRAIN/dataset/train/labels.cache


val: Scanning /home/qcienmed/mmr689/yolo-consumptions/datasets/bioview-lizards_TRAIN/dataset/validation/labels... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<00:00, 1221.82it/s]

val: New cache created: /home/qcienmed/mmr689/yolo-consumptions/datasets/bioview-lizards_TRAIN/dataset/validation/labels.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  1000/1000       2.3G     0.5656     0.3181     0.8355          4        640: 100%|██████████| 37/37 [00:11<00:00,  3.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/5 [00:00<?, ?it/s]/home/qcienmed/mmr689/yolo-consumptions/.venv/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.59it/s]


                   all        145        146      0.997      0.986      0.995      0.756

1000 epochs completed in 3.078 hours.
Optimizer stripped from /home/qcienmed/mmr689/yolo-consumptions/datasets/bioview-lizards_TRAIN/run/train/weights/last.pt, 6.4MB
Optimizer stripped from /home/qcienmed/mmr689/yolo-consumptions/datasets/bioview-lizards_TRAIN/run/train/weights/best.pt, 6.4MB

Validating /home/qcienmed/mmr689/yolo-consumptions/datasets/bioview-lizards_TRAIN/run/train/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.8.10 torch-2.3.0+cu121 CUDA:1 (NVIDIA RTX A6000, 48677MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


/home/qcienmed/mmr689/yolo-consumptions/.venv/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  5.16it/s]


                   all        145        146      0.997      0.986      0.995      0.755
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /home/qcienmed/mmr689/yolo-consumptions/datasets/bioview-lizards_TRAIN/run/train
